<a href="https://colab.research.google.com/github/eduion/AIOT/blob/main/HW6/HW6_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/chandrikadeb7/Face-Mask-Detection.git

Cloning into 'Face-Mask-Detection'...
remote: Enumerating objects: 4590, done.
remote: Total 4590 (delta 0), reused 0 (delta 0), pack-reused 4590 (from 1)
Receiving objects: 100% (4590/4590), 186.72 MiB | 16.35 MiB/s, done.
Resolving deltas: 100% (271/271), done.
Updating files: 100% (4155/4155), done.


In [4]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
# Prepare the data
def prepare_data():
    train_datagen = ImageDataGenerator(
        rescale=1.0/255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2  # Split training and validation sets
    )

    train_generator = train_datagen.flow_from_directory(
        'Face-Mask-Detection/dataset',  # Path to the dataset
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='training'
    )

    val_generator = train_datagen.flow_from_directory(
        'Face-Mask-Detection/dataset',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='validation'
    )

    return train_generator, val_generator

train_gen, val_gen = prepare_data()

Found 3274 images belonging to 2 classes.
Found 818 images belonging to 2 classes.


In [6]:
def build_vgg16_model():
    vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    vgg16_base.trainable = False  # Freeze convolutional base

    x = Flatten()(vgg16_base.output)
    x = Dense(256, activation='relu')(x)
    x = Dense(2, activation='softmax')(x)

    model = Model(inputs=vgg16_base.input, outputs=x)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_vgg16_model()

# Train the model
model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=3
)

# Save the model
model.save('/content/drive/My Drive/medical_mask_model.h5')

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 57/103 ━━━━━━━━━━━━━━━━━━━━ 26s 576ms/step - accuracy: 0.5698 - loss: 2.6731

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


103/103 ━━━━━━━━━━━━━━━━━━━━ 114s 914ms/step - accuracy: 0.6616 - loss: 1.8681 - val_accuracy: 0.9645 - val_loss: 0.1685
Epoch 2/3
103/103 ━━━━━━━━━━━━━━━━━━━━ 60s 554ms/step - accuracy: 0.9444 - loss: 0.1682 - val_accuracy: 0.9658 - val_loss: 0.1224
Epoch 3/3
103/103 ━━━━━━━━━━━━━━━━━━━━ 91s 642ms/step - accuracy: 0.9559 - loss: 0.1190 - val_accuracy: 0.9450 - val_loss: 0.1449


In [11]:
import requests
from PIL import Image
import numpy as np
from io import BytesIO

# 測試單張圖片的分類
def test_image(image_url, model, class_labels):
    """
    使用模型辨識從 URL 獲取的圖片。
    :param image_url: 圖片的網路地址
    :param model: 訓練好的 Keras 模型
    :param class_labels: 分類標籤列表（如 ["With Mask", "Without Mask"]）
    """
    try:
        # 從 URL 獲取圖片
        image_url = image_url.strip()
        response = requests.get(image_url)
        if response.status_code != 200:
            raise Exception("無法下載圖片，請檢查 URL 是否正確")

        # 讀取並處理圖片
        img = Image.open(BytesIO(response.content)).convert('RGB')
        img = img.resize((224, 224))  # 調整圖片尺寸
        img_array = np.array(img) / 255.0  # 歸一化像素值到 [0, 1]
        img_array = np.expand_dims(img_array, axis=0)  # 增加批次維度 (1, 224, 224, 3)

        # 使用模型進行預測
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions)  # 找到概率最大的類別
        confidence = predictions[0][predicted_class]  # 該類別的信心度

        # 輸出預測結果
        print(f"Predicted Class: {class_labels[predicted_class]} with confidence {confidence:.2f}")
    except Exception as e:
        print(f"錯誤: {e}")

# 測試模型
image_url = str("https://na.cx/i/eqzQJYw.jpg")
test_image(image_url, model, ['With Mask', 'Without Mask'])

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted Class: With Mask with confidence 0.99
